In [13]:
import pandas as pd

### IPS 
as "Indice de Position Social" is a index calculated from socio-professional categories of both parents and legal representative, revenues of parents, material situation, educational intrest and cultural implication.
It scales from 38 to 179 (no unit) and the actual formula to calculate it has not been made public. We possess the average IPS of nearly every highschools in France. 
IPS is a mean to observe and fight against school related inequalities.
### Ival
as "Indicateur de Valeur ajoutée" is a number (positive or negative, no unit) calculated from the actual result minus the expected result. This value shows if a school is capable of supporting their students from entry to graduation. 

In [14]:
#Dataframe containing ips of high schools in france
ips_df = pd.read_csv("ips.csv", delimiter=";")

#Dataframe containing exam results of high schools in france
res_df = pd.read_csv("results.csv", delimiter=";", low_memory=False)

In [15]:
ips_df.head(1)

,Rentrée scolaire,Académie,Code du département,Département,UAI,Nom de l'établissment,Code INSEE de la commune,Nom de la commune,Secteur,Type de lycée,IPS voie GT,IPS voie PRO,IPS Ensemble GT-PRO,Ecart-type de l'IPS voie GT,Ecart-type de l'IPS voie PRO
0,2018-2019,CRETEIL,77.0,SEINE-ET-MARNE,0771027Y,LYCEE POLYVALENT FREDERIC JOLIOT CURIE LYCEE D...,77152.0,DAMMARIE LES LYS,public,LPO,109.7,88.3,101.6,NaN,NaN


In [16]:
res_df.head(1)

,Etablissement,Annee,Ville,UAI,Code commune,Academie,Departement,Secteur,Presents - L,Presents - ES,...,Valeur ajoutee du taux de mentions - STMG,Valeur ajoutee du taux de mentions - STL,Valeur ajoutee du taux de mentions - ST2S,Valeur ajoutee du taux de mentions - TMD,Valeur ajoutee du taux de mentions - STHR,Presents - Gnle,Taux de reussite - Gnle,Valeur ajoutee du taux de réussite - Gnle,Taux de mentions - Gnle,Valeur ajoutée du taux de mentions - Gnle
0,LYCEE CHARLES PONCET (GENERAL ET TECHNO.),2021,CLUSES,0740017S,74081,GRENOBLE,HAUTE SAVOIE,public,NaN,NaN,...,17,NaN,NaN,NaN,NaN,279.0,95.0,-3,70.0,-1


In [17]:
#Taking out values = 'ND' since it is a string and does not appear frequently
res_df = res_df[res_df['Valeur ajoutee du taux de réussite - Gnle'] != 'ND']
#Taking out values taken before 2018 since ips_df does not have values (for exams which take place at the last year of the school year) before 2018
res_df = res_df[res_df["Annee"] >= 2017]

#Converting the year range "20XX(0) - 20XX(1)" to "20XX(1)"
ips_df["Rentrée scolaire"] = ips_df["Rentrée scolaire"].apply(lambda x: x.split(sep="-")[1])

In [18]:
#Convertting values for merge (avoid ValueError)
for col in ips_df.columns:
    ips_df[col] = ips_df[col].apply(str)

for col in res_df.columns:
    res_df[col] = res_df[col].apply(str)

In [19]:
ips_df = ips_df.rename(columns={"Rentrée scolaire":"Annee"})
#Inner to ensure all values are present for each high school per year
ips_res_df = pd.merge(ips_df, res_df, how="inner", on=["UAI", "Annee"])

In [24]:
#Creating a new df with only useful columns
ips_res_df = ips_res_df[["Annee", "Code region", "Code departement", "Code commune", "UAI", "Secteur_x", "Type de lycée", 
                        "IPS voie GT", "IPS voie PRO", "IPS Ensemble GT-PRO",
                        "Ecart-type de l'IPS voie GT", "Ecart-type de l'IPS voie PRO", 
                        "Presents - L", "Presents - ES", "Presents - S", "Presents - Toutes series",
                        "Taux de reussite - L", "Taux de reussite - ES", "Taux de reussite - S", "Taux de reussite - Toutes series",
                        "Taux de mentions - L", "Taux de mentions - ES", "Taux de mentions - S", "Taux de mentions - Toutes series",
                        "Effectif de seconde", "Effectif de premiere", "Effectif de terminale",
                        "Taux d'acces 2nde-bac", "Taux d'acces 1ere-bac", "Taux d'acces terminale-bac",
                        "Valeur ajoutee du taux d'acces 2nde-bac", "Valeur ajoutee du taux d'acces 1ere-bac", "Valeur ajoutee du taux d'acces terminale-bac",
                        "Valeur ajoutee du taux de reussite - Toutes series",
                        "Valeur ajoutee du taux de reussite - L", "Valeur ajoutee du taux de reussite - ES", "Valeur ajoutee du taux de reussite - S",
                        "Valeur ajoutee du taux de mentions - Toutes series",
                        "Valeur ajoutee du taux de mentions - L", "Valeur ajoutee du taux de mentions - ES", "Valeur ajoutee du taux de mentions - S",
                        "Presents - Gnle", "Taux de reussite - Gnle", "Taux de mentions - Gnle", 
                        "Valeur ajoutee du taux de réussite - Gnle", "Valeur ajoutée du taux de mentions - Gnle"
                        ]]

ips_res_df = ips_res_df.rename(columns={"Secteur_x":"Secteur"})

ips_res_df.to_csv("ips_results.csv")